In [167]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
from pykrx import stock
import pandas_datareader.data as pdr
from datetime import datetime, timedelta

In [168]:
df = pd.read_csv('.\s&p500\S&P 500 Historical Components & Changes(04-08-2024).csv')

In [169]:
df.head()

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


2000/01/01~2022/12/31까지 존재한 모든 주식을 열로 가지는 새로운 dataframe 생성

In [170]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter the data for the specified date range
filtered_data = df[(df['date'] >= '2006-01-01') & (df['date'] <= '2022-12-31')]

# Extract unique tickers over the filtered date range
unique_tickers = set()

for tickers in filtered_data['tickers']:
    unique_tickers.update(tickers.split(','))
# Create a new dataframe with the date range from 2006-01-01 to 2022-12-31
date_range = pd.date_range(start='2006-01-01', end='2022-12-31', freq='D')
new_df = pd.DataFrame(date_range, columns=['date'])
# Add a new column S&P500, SPX .INX, ^GSPC' after the 'date' column
new_df['^GSPC'] = np.nan
# Merge the new dataframe with the existing dataframe of unique tickers
sp500_df = pd.concat([new_df, pd.DataFrame(columns=sorted(unique_tickers))], axis=1)

In [171]:
sp500_df.head()

,date,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,...,XOM,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
unique_tickers

{'A',
 'AABA',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABI',
 'ABKFQ',
 'ABMD',
 'ABS',
 'ABT',
 'ACAS',
 'ACGL',
 'ACN',
 'ACS',
 'ACV',
 'ADBE',
 'ADCT',
 'ADI',
 'ADM',
 'ADP',
 'ADS',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AES',
 'AET',
 'AFL',
 'AGN',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'AKS',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALTR',
 'ALXN',
 'AMAT',
 'AMCC',
 'AMCR',
 'AMD',
 'AME',
 'AMG',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANDV',
 'ANDW',
 'ANET',
 'ANF',
 'ANRZQ',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'APC',
 'APCC',
 'APD',
 'APH',
 'APOL',
 'APTV',
 'ARE',
 'ARG',
 'ARNC',
 'ASH',
 'ASN',
 'ASO',
 'AT',
 'ATGE',
 'ATI',
 'ATO',
 'ATVI',
 'AV',
 'AVB',
 'AVGO',
 'AVP',
 'AVY',
 'AW',
 'AWK',
 'AXP',
 'AYE',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBBY',
 'BBT',
 'BBWI',
 'BBY',
 'BC',
 'BCR',
 'BDK',
 'BDX',
 'BEAM',
 'BEN',
 'BF.B',
 'BHF',
 'BHGE',
 'BIG',
 'BIIB',
 'BIO',
 'BJS',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BLL',
 'BLS',
 'BMC'

In [173]:
# Save the updated dataframe to a CSV file
output_file_path = '.\s&p500\s&p500_stock_data.csv'
sp500_df.to_csv(output_file_path, index=False)

만든 기본 틀에다가 yahoo finacne와 naver를 소스로 해서 정보 받아오기

In [174]:
sp500_df = pd.read_csv('.\s&p500\s&p500_stock_data.csv', parse_dates=['date'], index_col='date')

In [175]:
sp500_df.head()

,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XOM,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
ticker_data = yf.download('AAPL', start='2006-01-01', end='2022-12-31')
ticker_data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-01-03,2.585000,2.669643,2.580357,2.669643,2.257057,807234400
2006-01-04,2.683214,2.713571,2.660714,2.677500,2.263700,619603600
2006-01-05,2.672500,2.675000,2.633929,2.656429,2.245886,449422400
2006-01-06,2.687500,2.739286,2.662500,2.725000,2.303858,704457600
2006-01-09,2.740357,2.757143,2.705000,2.716071,2.296311,675040800


In [177]:
unique_tickers = sp500_df.columns
unique_tickers

Index(['^GSPC', 'A', 'AABA', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABI',
       'ABKFQ',
       ...
       'XOM', 'XRAY', 'XRX', 'XTO', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', length=891)

In [178]:
def update_ticker_data(ticker, df, start_date='2006-01-01', end_date='2022-12-31'):
    try:
        # Fetch historical market data using yfinance
        ticker_data = yf.download(ticker, start=start_date, end=end_date)
        if not ticker_data.empty:
            ticker_data = ticker_data['Close']
            # Align ticker_data to df index and update
            df[ticker] = ticker_data
        else:
            print(f"No data fetched for {ticker}")
            return df
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")
        return df
    
    return df

In [179]:
# Update the dataframe with the closing prices for each unique ticker
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    sp500_df = update_ticker_data(ticker, sp500_df)

Fetching data for ^GSPC...


[*********************100%%**********************]  1 of 1 completed


Fetching data for A...


[*********************100%%**********************]  1 of 1 completed


Fetching data for AABA...


...

[*********************100%%**********************]  1 of 1 completed


Fetching data for ZION...


[*********************100%%**********************]  1 of 1 completed


Fetching data for ZTS...


[*********************100%%**********************]  1 of 1 completed


In [180]:
sp500_df.head()

,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XOM,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-03,1268.800049,23.962805,NaN,37.450001,43.520000,2.669643,NaN,NaN,NaN,NaN,...,58.470001,27.014999,39.288540,NaN,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,NaN,39.200001,43.830002,2.677500,NaN,NaN,NaN,NaN,...,58.570000,27.264999,39.235836,NaN,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,NaN,39.709999,44.040001,2.656429,NaN,NaN,NaN,NaN,...,58.279999,26.955000,38.814228,NaN,NaN,17.670740,66.407768,42.529999,77.720001,NaN


In [181]:
# Save the updated dataframe to a CSV file
output_file_path = '.\s&p500\s&p500_stock_data.csv'
sp500_df.to_csv(output_file_path, index=True)

data 클랜징

In [182]:
sp500_df = pd.read_csv(output_file_path, parse_dates=['date'], index_col='date')
sp500_df.head()

,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XOM,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-03,1268.800049,23.962805,NaN,37.450001,43.520000,2.669643,NaN,NaN,NaN,NaN,...,58.470001,27.014999,39.288540,NaN,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,NaN,39.200001,43.830002,2.677500,NaN,NaN,NaN,NaN,...,58.570000,27.264999,39.235836,NaN,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,NaN,39.709999,44.040001,2.656429,NaN,NaN,NaN,NaN,...,58.279999,26.955000,38.814228,NaN,NaN,17.670740,66.407768,42.529999,77.720001,NaN


In [183]:
#모든 행이 NA면 제거(공휴일)
sp500_df = sp500_df.dropna(how='all')

In [184]:
sp500_df.head()

,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XOM,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,1268.800049,23.962805,NaN,37.450001,43.520000,2.669643,NaN,NaN,NaN,NaN,...,58.470001,27.014999,39.288540,NaN,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,NaN,39.200001,43.830002,2.677500,NaN,NaN,NaN,NaN,...,58.570000,27.264999,39.235836,NaN,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,NaN,39.709999,44.040001,2.656429,NaN,NaN,NaN,NaN,...,58.279999,26.955000,38.814228,NaN,NaN,17.670740,66.407768,42.529999,77.720001,NaN
2006-01-06,1285.449951,24.785408,NaN,39.000000,43.990002,2.725000,NaN,NaN,NaN,NaN,...,59.430000,27.375000,38.577076,NaN,NaN,17.620417,66.067963,44.119999,78.529999,NaN
2006-01-09,1290.150024,24.713877,NaN,38.610001,44.560001,2.716071,NaN,NaN,NaN,NaN,...,59.400002,27.645000,39.104084,NaN,NaN,17.757010,68.407768,44.790001,77.879997,NaN


In [185]:
#모든 값이 Nan인 종목제거(해당기간에 없었음)
sp500_df = sp500_df.dropna(axis=1, how='all')
sp500_df.head()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,1268.800049,23.962805,37.450001,43.520000,2.669643,NaN,9.44,NaN,18.976089,6.120000,...,18.570000,58.470001,27.014999,39.288540,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,39.200001,43.830002,2.677500,NaN,9.62,NaN,19.004877,6.151111,...,18.660000,58.570000,27.264999,39.235836,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,39.709999,44.040001,2.656429,NaN,9.55,NaN,19.249577,6.061111,...,18.650000,58.279999,26.955000,38.814228,NaN,17.670740,66.407768,42.529999,77.720001,NaN
2006-01-06,1285.449951,24.785408,39.000000,43.990002,2.725000,NaN,9.75,NaN,19.619022,6.173333,...,18.719999,59.430000,27.375000,38.577076,NaN,17.620417,66.067963,44.119999,78.529999,NaN
2006-01-09,1290.150024,24.713877,38.610001,44.560001,2.716071,NaN,10.15,NaN,20.348318,6.116667,...,18.670000,59.400002,27.645000,39.104084,NaN,17.757010,68.407768,44.790001,77.879997,NaN


In [186]:
# 열을 검사하여 5열 연속 비어있는 열의 경우 -1로 변경
#향후 보간법 적용을 원활히 하기 위함
# 결측치를 -1로 변경하는 함수 정의
def replace_long_nan_streaks(series, threshold=5):
    count = 0
    for i in range(len(series)):
        if pd.isna(series[i]):
            count += 1
        else:
            if count >= threshold:
                series[i-count:i] = -1
            count = 0
    if count >= threshold:  # 끝에 도달했을 때 연속된 NaN이 threshold 이상인 경우
        series[-count:] = -1
    return series

# 각 열에 대해 결측치를 -1로 변경
for col in sp500_df.columns:
    sp500_df[col] = replace_long_nan_streaks(sp500_df[col])

In [187]:
sp500_df.isna().sum().sum()

3193

In [188]:
# 선형 보간법을 사용하여 NaN 값 채우기
sp500_df = sp500_df.interpolate(method='linear')

In [189]:
sp500_df.isna().sum().sum()

0

In [190]:
#다시 Nan으로 변경
sp500_df.replace(-1, np.nan, inplace=True)

In [192]:
sp500_df.head()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,1268.800049,23.962805,37.450001,43.520000,2.669643,NaN,9.44,NaN,18.976089,6.120000,...,18.570000,58.470001,27.014999,39.288540,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,39.200001,43.830002,2.677500,NaN,9.62,NaN,19.004877,6.151111,...,18.660000,58.570000,27.264999,39.235836,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,39.709999,44.040001,2.656429,NaN,9.55,NaN,19.249577,6.061111,...,18.650000,58.279999,26.955000,38.814228,NaN,17.670740,66.407768,42.529999,77.720001,NaN
2006-01-06,1285.449951,24.785408,39.000000,43.990002,2.725000,NaN,9.75,NaN,19.619022,6.173333,...,18.719999,59.430000,27.375000,38.577076,NaN,17.620417,66.067963,44.119999,78.529999,NaN
2006-01-09,1290.150024,24.713877,38.610001,44.560001,2.716071,NaN,10.15,NaN,20.348318,6.116667,...,18.670000,59.400002,27.645000,39.104084,NaN,17.757010,68.407768,44.790001,77.879997,NaN


In [191]:
# Save the updated dataframe to a CSV file
sp500_df.to_csv('.\s&p500\clean_s&p500_stock_data.csv', index=True)